In [113]:
import warnings
warnings.filterwarnings('ignore')

In [114]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [115]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

# Read the CSV and Perform Basic Data Cleaning

In [116]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [117]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [118]:
pd.set_option("display.max_rows", 89)
df.select_dtypes(include = ["object"])

,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,initial_list_status,next_pymnt_d,application_type,hardship_flag,debt_settlement_flag
0,RENT,Source Verified,Mar-2019,low_risk,n,w,May-2019,Individual,N,N
1,MORTGAGE,Verified,Mar-2019,low_risk,n,w,May-2019,Individual,N,N
2,MORTGAGE,Verified,Mar-2019,low_risk,n,w,May-2019,Individual,N,N
3,RENT,Verified,Mar-2019,low_risk,n,w,May-2019,Individual,N,N
4,MORTGAGE,Not Verified,Mar-2019,low_risk,n,w,May-2019,Individual,N,N
...,...,...,...,...,...,...,...,...,...,...
68812,RENT,Source Verified,Jan-2019,low_risk,n,w,May-2019,Individual,N,N
68813,RENT,Not Verified,Jan-2019,low_risk,n,w,May-2019,Individual,N,N
68814,MORTGAGE,Source Verified,Jan-2019,low_risk,n,w,May-2019,Individual,N,N
68815,MORTGAGE,Verified,Jan-2019,low_risk,n,f,May-2019,Individual,N,N


In [30]:
#df['issue_d'].unique()

In [31]:
#convert to datetime object
# dates = [(df['issue_d'] == 'Mar-2019'), (df['issue_d'] == 'Feb-2019'), (df['issue_d'] == 'Jan-2019')]
# values = ['3', '2', '1']
# df['issue_dates'] = np.select(dates, values)
# df

In [32]:
# df.drop(columns=['issue_d'], inplace=True)
# df

In [68]:
df['pymnt_plan'].unique()

array(['n'], dtype=object)

In [69]:
df['hardship_flag'].unique()

array(['N'], dtype=object)

In [70]:
df['debt_settlement_flag'].unique()

array(['N'], dtype=object)

In [119]:
df.drop(columns=['pymnt_plan', 'hardship_flag', 'debt_settlement_flag'], inplace=True)
df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,dti,delinq_2yrs,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,27.24,0.0,...,0.0,3.0,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,20.23,0.0,...,0.0,6.0,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,24.26,0.0,...,0.0,4.0,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,31.44,0.0,...,0.0,1.0,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,18.76,0.0,...,0.0,1.0,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,RENT,26000.0,Source Verified,Jan-2019,low_risk,9.60,0.0,...,0.0,3.0,80.0,0.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0
68813,12000.0,0.2727,368.37,RENT,63000.0,Not Verified,Jan-2019,low_risk,29.07,0.0,...,0.0,5.0,96.2,0.0,0.0,0.0,87939.0,60350.0,13500.0,62939.0
68814,5000.0,0.1992,185.62,MORTGAGE,52000.0,Source Verified,Jan-2019,low_risk,14.86,0.0,...,0.0,3.0,100.0,0.0,1.0,0.0,30592.0,18611.0,3600.0,18492.0
68815,40000.0,0.0646,1225.24,MORTGAGE,520000.0,Verified,Jan-2019,low_risk,9.96,0.0,...,0.0,3.0,98.2,12.5,0.0,0.0,1033574.0,95958.0,100800.0,78634.0


In [120]:
pd.set_option("display.max_rows", 89)
df.select_dtypes(include = ["object"])

,home_ownership,verification_status,issue_d,loan_status,initial_list_status,next_pymnt_d,application_type
0,RENT,Source Verified,Mar-2019,low_risk,w,May-2019,Individual
1,MORTGAGE,Verified,Mar-2019,low_risk,w,May-2019,Individual
2,MORTGAGE,Verified,Mar-2019,low_risk,w,May-2019,Individual
3,RENT,Verified,Mar-2019,low_risk,w,May-2019,Individual
4,MORTGAGE,Not Verified,Mar-2019,low_risk,w,May-2019,Individual
...,...,...,...,...,...,...,...
68812,RENT,Source Verified,Jan-2019,low_risk,w,May-2019,Individual
68813,RENT,Not Verified,Jan-2019,low_risk,w,May-2019,Individual
68814,MORTGAGE,Source Verified,Jan-2019,low_risk,w,May-2019,Individual
68815,MORTGAGE,Verified,Jan-2019,low_risk,f,May-2019,Individual


In [121]:
df['issue_d'] = pd.to_datetime(df['issue_d'])

In [122]:
df['next_pymnt_d'] = pd.to_datetime(df['next_pymnt_d'])

In [75]:
pd.set_option("display.max_rows", 89)
df.select_dtypes(include = ["object"])

,home_ownership,verification_status,loan_status,initial_list_status,application_type
0,RENT,Source Verified,low_risk,w,Individual
1,MORTGAGE,Verified,low_risk,w,Individual
2,MORTGAGE,Verified,low_risk,w,Individual
3,RENT,Verified,low_risk,w,Individual
4,MORTGAGE,Not Verified,low_risk,w,Individual
...,...,...,...,...,...
68812,RENT,Source Verified,low_risk,w,Individual
68813,RENT,Not Verified,low_risk,w,Individual
68814,MORTGAGE,Source Verified,low_risk,w,Individual
68815,MORTGAGE,Verified,low_risk,f,Individual


In [76]:
df['initial_list_status'].unique()

array(['w', 'f'], dtype=object)

# Split the Data into Training and Testing

In [123]:
target_encoded = pd.get_dummies(df, columns=["home_ownership", "verification_status", 
                                             "initial_list_status", "issue_d", "next_pymnt_d", "application_type"])
target_encoded.head(20)

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,verification_status_Verified,initial_list_status_f,initial_list_status_w,issue_d_2019-01-01 00:00:00,issue_d_2019-02-01 00:00:00,issue_d_2019-03-01 00:00:00,next_pymnt_d_2019-04-01 00:00:00,next_pymnt_d_2019-05-01 00:00:00,application_type_Individual,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,0,0,1,0,0,1,0,1,1,0
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,0,1,0,0,1,0,1,1,0
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,0,1,0,0,1,0,1,1,0
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,1,0,1,0,0,1,0,1,1,0
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,0,0,1,0,0,1,0,1,1,0
5,25000.0,0.1797,634.43,90000.0,low_risk,18.49,1.0,1.0,14.0,0.0,...,0,0,1,0,0,1,0,1,1,0
6,20400.0,0.2000,540.48,51315.0,low_risk,25.63,0.0,2.0,8.0,1.0,...,0,0,1,0,0,1,0,1,1,0
7,10000.0,0.1557,349.46,66000.0,low_risk,9.00,0.0,0.0,11.0,0.0,...,0,0,1,0,0,1,0,1,1,0
8,14000.0,0.0881,443.96,45000.0,low_risk,10.24,0.0,1.0,9.0,0.0,...,0,1,0,0,0,1,0,1,1,0
9,10000.0,0.1474,345.39,63295.0,low_risk,39.34,0.0,0.0,27.0,0.0,...,1,0,1,0,0,1,0,1,1,0


In [124]:
# Create our features
X = target_encoded.copy()
X = target_encoded.drop(columns="loan_status")


# Create our target
y = target_encoded["loan_status"]

In [79]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,initial_list_status_f,initial_list_status_w,issue_d_2019-01-01 00:00:00,issue_d_2019-02-01 00:00:00,issue_d_2019-03-01 00:00:00,next_pymnt_d_2019-04-01 00:00:00,next_pymnt_d_2019-05-01 00:00:00,application_type_Individual,application_type_Joint App
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.148001,0.123879,0.876121,0.451066,0.371696,0.177238,0.383161,0.616839,0.860340,0.139660
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.355104,0.329446,0.329446,0.497603,0.483261,0.381873,0.486161,0.486161,0.346637,0.346637
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [80]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [125]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(51612, 92)
(17205, 92)
(51612,)
(17205,)


In [126]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler; standardizing 
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [127]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1) 
#fit on training model
brf_model.fit(X_train_scaled, y_train)
predictions = brf_model.predict(X_test_scaled)

In [128]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, predictions)
acc_score

0.7809842987338959

In [129]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[   69,    32],
       [ 2073, 15031]], dtype=int64)

In [130]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.68      0.88      0.06      0.77      0.59       101
   low_risk       1.00      0.88      0.68      0.93      0.77      0.61     17104

avg / total       0.99      0.88      0.68      0.93      0.77      0.61     17205



In [131]:
# List the features sorted in descending order by feature importance
sorted(zip(brf_model.feature_importances_, X.columns), reverse=True)

[(0.07089969080639422, 'total_rec_int'),
 (0.06303985121482761, 'last_pymnt_amnt'),
 (0.05918013901194053, 'total_pymnt'),
 (0.055797057990852245, 'total_pymnt_inv'),
 (0.05311967863216865, 'total_rec_prncp'),
 (0.031470590870623885, 'int_rate'),
 (0.02510658586920422, 'issue_d_2019-01-01 00:00:00'),
 (0.02116832850250925, 'mths_since_recent_inq'),
 (0.017961014927095433, 'dti'),
 (0.017881680892978986, 'out_prncp'),
 (0.016975699697096343, 'installment'),
 (0.016505072835350583, 'annual_inc'),
 (0.016303373757178084, 'out_prncp_inv'),
 (0.015334122424167807, 'mths_since_rcnt_il'),
 (0.015298924958769304, 'max_bal_bc'),
 (0.014878966760591207, 'tot_cur_bal'),
 (0.014682465516985252, 'avg_cur_bal'),
 (0.014676453885483033, 'loan_amnt'),
 (0.014099147656051051, 'total_bal_ex_mort'),
 (0.013584279744153663, 'mths_since_recent_bc'),
 (0.013546776447664312, 'mo_sin_old_il_acct'),
 (0.013425021520425488, 'il_util'),
 (0.01322879716976006, 'bc_open_to_buy'),
 (0.01313969053431428, 'bc_util'),

### Easy Ensemble AdaBoost Classifier

In [134]:
#to resolve 'EasyEnsembleClassifier' object has no attribute 'n_features_in_' error but didn't work
!pip3 install scikit-learn==1.0 -U

In [140]:
# Train the EasyEnsembleClassifier
#from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.ensemble import AdaBoostClassifier
easy_model = AdaBoostClassifier(n_estimators=100, random_state=1) 
easy_model.fit(X_train_scaled, y_train)
y_predictions = easy_model.predict(X_test_scaled)

In [143]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, predictions)
acc_score

0.7809842987338959

In [145]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[   69,    32],
       [ 2073, 15031]], dtype=int64)

In [147]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.68      0.88      0.06      0.77      0.59       101
   low_risk       1.00      0.88      0.68      0.93      0.77      0.61     17104

avg / total       0.99      0.88      0.68      0.93      0.77      0.61     17205

